In [148]:
import pandas as pd
import requests
import hvplot.pandas

from api_keys import geoapify_key

In [149]:
city_data_df = pd.read_csv("output_data/cities.csv")

city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


In [150]:
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size= 'Humidity'
)

In [151]:
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

In [152]:
ideal_df = city_data_df[(city_data_df['Humidity'] <= 50) & (city_data_df['Max Temp'] >= 16) & (city_data_df['Max Temp'] <= 25)]

In [153]:
ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
86,86,rafaela,-31.2503,-61.4867,21.10,42,100,1.79,AR,1666108109
102,102,rio gallegos,-51.6226,-69.2181,20.03,26,75,8.23,AR,1666108296
106,106,palencia,42.4167,-4.5000,24.40,30,40,8.66,ES,1666108219
114,114,yidu,24.1614,115.1328,19.73,27,99,2.78,CN,1666108303
229,229,san cristobal,7.7669,-72.2250,22.10,45,46,0.57,VE,1666108372
239,239,rawson,-43.3002,-65.1023,23.74,25,2,13.52,AR,1666108102
296,296,jalu,29.0331,21.5482,22.33,37,25,3.89,LY,1666108413
305,305,kingman,35.1894,-114.0530,20.07,49,5,4.02,US,1666108417
313,313,tandil,-37.3217,-59.1332,21.63,40,75,6.10,AR,1666108422


In [154]:
hotel_df=ideal_df.copy()

In [155]:
hotel_df['Hotel Name']=''

In [156]:
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228,
86,86,rafaela,-31.2503,-61.4867,21.10,42,100,1.79,AR,1666108109,
102,102,rio gallegos,-51.6226,-69.2181,20.03,26,75,8.23,AR,1666108296,
106,106,palencia,42.4167,-4.5000,24.40,30,40,8.66,ES,1666108219,
114,114,yidu,24.1614,115.1328,19.73,27,99,2.78,CN,1666108303,
229,229,san cristobal,7.7669,-72.2250,22.10,45,46,0.57,VE,1666108372,
239,239,rawson,-43.3002,-65.1023,23.74,25,2,13.52,AR,1666108102,
296,296,jalu,29.0331,21.5482,22.33,37,25,3.89,LY,1666108413,
305,305,kingman,35.1894,-114.0530,20.07,49,5,4.02,US,1666108417,
313,313,tandil,-37.3217,-59.1332,21.63,40,75,6.10,AR,1666108422,


In [160]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": api_keys,
    "bias": f"proximity:{lon},{lat}",
    f'filter=circle:-87.770231,41.878968,5000
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row.loc['Lat']
    lon = row.loc['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"custom_loc:{lat},{lon}:2000"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
faya - nearest hotel: No hotel found
rafaela - nearest hotel: No hotel found
rio gallegos - nearest hotel: No hotel found
palencia - nearest hotel: No hotel found
yidu - nearest hotel: No hotel found
san cristobal - nearest hotel: No hotel found
rawson - nearest hotel: No hotel found
jalu - nearest hotel: No hotel found
kingman - nearest hotel: No hotel found
tandil - nearest hotel: No hotel found
viseu - nearest hotel: No hotel found
ribeira grande - nearest hotel: No hotel found
vryburg - nearest hotel: No hotel found
chivilcoy - nearest hotel: No hotel found


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228,No hotel found
86,86,rafaela,-31.2503,-61.4867,21.10,42,100,1.79,AR,1666108109,No hotel found
102,102,rio gallegos,-51.6226,-69.2181,20.03,26,75,8.23,AR,1666108296,No hotel found
106,106,palencia,42.4167,-4.5000,24.40,30,40,8.66,ES,1666108219,No hotel found
114,114,yidu,24.1614,115.1328,19.73,27,99,2.78,CN,1666108303,No hotel found
229,229,san cristobal,7.7669,-72.2250,22.10,45,46,0.57,VE,1666108372,No hotel found
239,239,rawson,-43.3002,-65.1023,23.74,25,2,13.52,AR,1666108102,No hotel found
296,296,jalu,29.0331,21.5482,22.33,37,25,3.89,LY,1666108413,No hotel found
305,305,kingman,35.1894,-114.0530,20.07,49,5,4.02,US,1666108417,No hotel found
313,313,tandil,-37.3217,-59.1332,21.63,40,75,6.10,AR,1666108422,No hotel found


In [161]:
name_address

{'statusCode': 400,
 'error': 'Bad Request',
 'message': '"filter[0]" does not match any of the allowed types'}